In [42]:
# importing libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from dython import nominal
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from pandas_profiling import ProfileReport
#from imblearn.over_sampling import SMOTE
import xgboost as xgb

In [43]:
icu_df = pd.read_csv('/Users/priyankasharma/Desktop/training_v2.csv') 
print (icu_df.head(10) )
icu_test_df = pd.read_csv('/Users/priyankasharma/Desktop/training_v2.csv')

   encounter_id  patient_id  hospital_id  hospital_death   age    bmi  \
0         66154       25312          118               0  68.0  22.73   
1        114252       59342           81               0  77.0  27.42   
2        119783       50777          118               0  25.0  31.95   
3         79267       46918          118               0  81.0  22.64   
4         92056       34377           33               0  19.0    NaN   
5         33181       74489           83               0  67.0  27.56   
6         82208       49526           83               0  59.0  57.45   
7        120995       50129           33               0  70.0    NaN   
8         80471       10577          118               1  45.0    NaN   
9         42871       90749          118               0  50.0  25.71   

   elective_surgery  ethnicity gender  height hospital_admit_source  \
0                 0  Caucasian      M   180.3                 Floor   
1                 0  Caucasian      F   160.0         

In [44]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [45]:
icu_df.describe()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,91713.000000,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90379.000000,91713.000000,91713.000000,91713.0,88993.000000,37334.000000,90051.000000,90612.000000,91713.000000,90998.000000,33579.000000,72451.00000,72860.000000,20845.000000,89812.000000,89812.000000,90676.000000,89812.000000,80677.000000,90835.000000,71835.000000,90998.000000,90719.000000,20845.000000,20845.000000,20845.000000,20845.000000,90479.000000,73113.000000,87605.000000,42715.000000,90998.000000,69701.000000,23729.000000,23729.000000,91548.000000,91548.000000,90673.000000,90673.000000,91568.000000,91568.000000,23936.000000,23936.000000,91493.000000,91493.000000,90234.000000,90234.000000,91328.000000,91328.000000,91380.000000,91380.000000,23754.000000,23754.000000,91554.000000,91554.00000,90686.000000,90686.000000,89389.000000,89389.000000,16785.000000,16785.000000,88094.000000,88094.000000,84363.000000,84363.000000,88923.000000,88923.000000,16869.000000,16869.000000,87074.000000,87074.000000,82629.000000,82629.000000,87356.000000,87356.000000,87528.000000,87528.000000,16798.000000,

In [46]:
icu_df.dtypes

encounter_id                       int64
patient_id                         int64
hospital_id                        int64
hospital_death                     int64
age                              float64
bmi                              float64
elective_surgery                   int64
ethnicity                         object
gender                            object
height                           float64
hospital_admit_source             object
icu_admit_source                  object
icu_id                             int64
icu_stay_type                     object
icu_type                          object
pre_icu_los_days                 float64
readmission_status                 int64
weight                           float64
albumin_apache                   float64
apache_2_diagnosis               float64
apache_3j_diagnosis              float64
apache_post_operative              int64
arf_apache                       float64
bilirubin_apache                 float64
bun_apache      

In [47]:
# profile = ProfileReport(icu_df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile
# profile.to_file(output_file="output_entire_data.html")

In [48]:
# profile = ProfileReport(icu_df.loc[icu_df['hospital_death'] == 1], title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile
# profile.to_file(output_file="output_deaths_data.html")

In [49]:
# profile = ProfileReport(icu_df.loc[icu_df['hospital_death'] == 0], title='Pandas Profiling Report for survived', html={'style':{'full_width':True}})
# profile.to_file(output_file="output_survived_data.html")

In [50]:
#Boxplot
box_var_y = "d1_lactate_max" #@param ['age', 'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height', 'hospital_admit_source', 'icu_admit_source', 'icu_id', 'icu_stay_type', 'icu_type', 'pre_icu_los_days', 'readmission_status', 'weight', 'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'apache_post_operative', 'arf_apache', 'bilirubin_apache', 'bun_apache', 'creatinine_apache', 'fio2_apache', 'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache', 'hematocrit_apache', 'intubated_apache', 'map_apache', 'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache', 'resprate_apache', 'sodium_apache', 'temp_apache', 'urineoutput_apache', 'ventilated_apache', 'wbc_apache', 'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min', 'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_invasive_max', 'd1_sysbp_invasive_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_invasive_max', 'h1_diasbp_invasive_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_invasive_max', 'h1_mbp_invasive_min', 'h1_mbp_max', 'h1_mbp_min', 'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_max', 'h1_spo2_min', 'h1_sysbp_invasive_max', 'h1_sysbp_invasive_min', 'h1_sysbp_max', 'h1_sysbp_min', 'h1_sysbp_noninvasive_max', 'h1_sysbp_noninvasive_min', 'h1_temp_max', 'h1_temp_min', 'd1_albumin_max', 'd1_albumin_min', 'd1_bilirubin_max', 'd1_bilirubin_min', 'd1_bun_max', 'd1_bun_min', 'd1_calcium_max', 'd1_calcium_min', 'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min', 'd1_hco3_max', 'd1_hco3_min', 'd1_hemaglobin_max', 'd1_hemaglobin_min', 'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_inr_max', 'd1_inr_min', 'd1_lactate_max', 'd1_lactate_min', 'd1_platelets_max', 'd1_platelets_min', 'd1_potassium_max', 'd1_potassium_min', 'd1_sodium_max', 'd1_sodium_min', 'd1_wbc_max', 'd1_wbc_min', 'h1_albumin_max', 'h1_albumin_min', 'h1_bilirubin_max', 'h1_bilirubin_min', 'h1_bun_max', 'h1_bun_min', 'h1_calcium_max', 'h1_calcium_min', 'h1_creatinine_max', 'h1_creatinine_min', 'h1_glucose_max', 'h1_glucose_min', 'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max', 'h1_hemaglobin_min', 'h1_hematocrit_max', 'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min', 'h1_lactate_max', 'h1_lactate_min', 'h1_platelets_max', 'h1_platelets_min', 'h1_potassium_max', 'h1_potassium_min', 'h1_sodium_max', 'h1_sodium_min', 'h1_wbc_max', 'h1_wbc_min', 'd1_arterial_pco2_max', 'd1_arterial_pco2_min', 'd1_arterial_ph_max', 'd1_arterial_ph_min', 'd1_arterial_po2_max', 'd1_arterial_po2_min', 'd1_pao2fio2ratio_max', 'd1_pao2fio2ratio_min', 'h1_arterial_pco2_max', 'h1_arterial_pco2_min', 'h1_arterial_ph_max', 'h1_arterial_ph_min', 'h1_arterial_po2_max', 'h1_arterial_po2_min', 'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', 'aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis', 'apache_3j_bodysystem', 'apache_2_bodysystem']
box_var_x = "hospital_death"

px.box(icu_df, y=box_var_y, x=box_var_x)

In [51]:
# corr_matrix = icu_df.corr(method='spearman')
# corr_matrix.to_excel("corr_matrix.xlsx")

ventilated_apache ~ gcs_verbal_apache = -0.50832186793227
d1_pao2fio2ratio_min ~ fio2_apache = -0.51
apache_4a_icu_death_prob ~ gcs_verbal_apache = -0.52
h1_arterial_ph_min ~ h1_arterial_pco2_max = -0.516
h1_arterial_ph_max ~ h1_arterial_pco2_min = -0.50

bmi ~ weight = 0.87
apache_3j_diagnosis ~ elective_surgery = 0.60
apache_3j_diagnosis ~ apache_post_operative = 0.69
apache_post_operative ~ elective_surgery = 0.90
albumin_apache ~ d1_albumin_max = 0.955
albumin_apache ~ d1_albumin_min = 0.96
albumin_apache ~ d1_calcium_min = 0.50
albumin_apache ~ h1_albumin_max = 0.88
albumin_apache ~ h1_albumin_min = 0.88
bilirubin_apache ~ d1_bilirubin_max = 0.98
bilirubin_apache ~ d1_bilirubin_min = 0.96
bilirubin_apache ~ h1_bilirubin_max = 0.93
bilirubin_apache ~ h1_bilirubin_min = 0.93
bun_apache ~ creatinine_apache = 0.73
bun_apache ~ d1_bun_max = 0.99
bun_apache ~ d1_bun_min = 0.97
bun_apache ~ d1_creatinine_max = 0.74
bun_apache ~ d1_creatinine_min = 0.72
bun_apache ~ h1_bun_max = 0.96
bun_apache ~ h1_bun_min = 0.96
bun_apache ~ h1_creatinine_max = 0.74
bun_apache ~ h1_creatinine_min = 0.74
creatinine_apache ~ d1_bun_max = 0.73
creatinine_apache ~ d1_bun_min = 0.725
creatinine_apache ~ d1_creatinine_max = 0.97
creatinine_apache ~ d1_creatinine_min = 0.98
creatinine_apache ~ h1_bun_max = 0.70
creatinine_apache ~ h1_bun_min = 0.70
creatinine_apache ~ h1_creatinine_max = 0.95
creatinine_apache ~ h1_creatinine_min = 0.95
gcs_eyes_apache ~ gcs_motor_apache = 0.72
gcs_eyes_apache ~ gcs_verbal_apache = 0.71
gcs_verbal_apache ~ gcs_motor_apache = 0.73
glucose_apache ~ d1_glucose_max = 0.81
glucose_apache ~ d1_glucose_min = 0.62
glucose_apache ~ h1_glucose_max = 0.73
glucose_apache ~ h1_glucose_min = 0.72
heart_rate_apache ~ d1_heartrate_max = 0.84
heart_rate_apache ~ d1_heartrate_min = 0.60
heart_rate_apache ~ h1_heartrate_max = 0.74
heart_rate_apache ~ h1_heartrate_min = 0.70
hematocrit_apache ~ d1_hemaglobin_max = 0.88
hematocrit_apache ~ d1_hemaglobin_min = 0.95
hematocrit_apache ~ d1_hematocrit_max = 0.92
hematocrit_apache ~ d1_hematocrit_min = 0.97
hematocrit_apache ~ h1_hemaglobin_max = 0.83
hematocrit_apache ~ h1_hemaglobin_min = 0.84
hematocrit_apache ~ h1_hematocrit_max = 0.86
hematocrit_apache ~ h1_hematocrit_min = 0.86
intubated_apache ~ ventilated_apache = 0.60
map_apache ~ d1_mbp_min = 0.53
map_apache ~ d1_mbp_noninvasive_min = 0.53
map_apache ~  h1_mbp_noninvasive_min = 0.50
paco2_apache ~ paco2_for_ph_apache = 1
paco2_apache ~ d1_arterial_pco2_max = 0.82
paco2_apache ~ d1_arterial_pco2_min = 0.80
paco2_apache ~ h1_arterial_pco2_max = 0.71
paco2_apache ~ h1_arterial_pco2_min = 0.70
paco2_for_ph_apache ~ d1_arterial_pco2_max = 0.82
paco2_for_ph_apache ~ d1_arterial_pco2_min = 0.80
paco2_for_ph_apache ~ h1_arterial_pco2_max = 0.71
paco2_for_ph_apache ~ h1_arterial_pco2_min = 0.70
pao2_apache ~ 
d1_potassium_max ~ h1_potassium_max = 0.72
d1_potassium_min ~ h1_potassium_max = 0.71
d1_potassium_max ~ h1_potassium_min = 0.68
d1_potassium_min ~ h1_potassium_min = 0.74
h1_potassium_max ~ h1_potassium_min = 0.96

......tba

In [52]:
icu_df2 = icu_df.copy(deep=True)
icu_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Columns: 186 entries, encounter_id to apache_2_bodysystem
dtypes: float64(170), int64(8), object(8)
memory usage: 130.1+ MB


# Handling Missing Values: icu_df2 : convert all NA's to zero

In [53]:
icu_df2.loc[icu_df2['pre_icu_los_days'] <= 0, 'pre_icu_los_days'] = np.NaN
icu_df2.loc[icu_df2['apache_4a_hospital_death_prob'] == -1, 'apache_4a_hospital_death_prob'] = np.NaN
icu_df2.loc[icu_df2['apache_4a_icu_death_prob'] == -1, 'apache_4a_icu_death_prob'] = np.NaN
# print(icu_df2['pre_icu_los_days'].min())
#icu_df2['apache_4a_icu_death_prob'].min()

In [54]:
icu_df2 = icu_df2.replace(to_replace=np.NaN, value=0)
print(type(icu_df2))

<class 'pandas.core.frame.DataFrame'>


In [55]:
X_col_dropped = ['encounter_id','patient_id','hospital_id','icu_id','hospital_death','bmi','readmission_status']
X = icu_df2.drop(labels=X_col_dropped, axis=1)
X = X.select_dtypes(exclude=['object'])
print(X.dtypes)
X_col_names = X.columns.values
X_col_names


age                              float64
elective_surgery                   int64
height                           float64
pre_icu_los_days                 float64
weight                           float64
albumin_apache                   float64
apache_2_diagnosis               float64
apache_3j_diagnosis              float64
apache_post_operative              int64
arf_apache                       float64
bilirubin_apache                 float64
bun_apache                       float64
creatinine_apache                float64
fio2_apache                      float64
gcs_eyes_apache                  float64
gcs_motor_apache                 float64
gcs_unable_apache                float64
gcs_verbal_apache                float64
glucose_apache                   float64
heart_rate_apache                float64
hematocrit_apache                float64
intubated_apache                 float64
map_apache                       float64
paco2_apache                     float64
paco2_for_ph_apa

array(['age', 'elective_surgery', 'height', 'pre_icu_los_days', 'weight',
       'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis',
       'apache_post_operative', 'arf_apache', 'bilirubin_apache',
       'bun_apache', 'creatinine_apache', 'fio2_apache',
       'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache',
       'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache',
       'hematocrit_apache', 'intubated_apache', 'map_apache',
       'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache',
       'resprate_apache', 'sodium_apache', 'temp_apache',
       'urineoutput_apache', 'ventilated_apache', 'wbc_apache',
       'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min',
       'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max',
       'd1_diasbp_noninvasive_min', 'd1_heartrate_max',
       'd1_heartrate_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min',
       'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max',
       'd1_mbp_noninva

In [56]:
y = icu_df2.loc[:,'hospital_death']
print(y.unique())
y

[0 1]


0        0
1        0
2        0
3        0
4        0
        ..
91708    0
91709    0
91710    0
91711    0
91712    0
Name: hospital_death, Length: 91713, dtype: int64

Train & Val split (70:30)

In [57]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))

64199 64199
27514 27514


In [58]:
rfc = RandomForestClassifier(n_jobs=-1)
fitted_model = rfc.fit(X_train, y_train)
y_pred = fitted_model.predict(X_val)
pTot = accuracy_score(y_true=y_val, y_pred=y_pred)
pTot

/Users/priyankasharma/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



0.9261466889583485

In [59]:
cm = confusion_matrix(y_val, y_pred)
print(cm)
report = classification_report(y_val, y_pred)
print (report) #for each class prints: precision  recall  f1-score   support

[[24870   269]
 [ 1763   612]]
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     25139
           1       0.69      0.26      0.38      2375

   micro avg       0.93      0.93      0.93     27514
   macro avg       0.81      0.62      0.67     27514
weighted avg       0.91      0.93      0.91     27514



In [60]:
rfc = RandomForestClassifier(class_weight='balanced', n_jobs=-1)
fitted_model = rfc.fit(X_train, y_train)
y_pred = fitted_model.predict(X_val)
pTot = accuracy_score(y_true=y_val, y_pred=y_pred)
pTot

/Users/priyankasharma/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



0.9210947154176056

In [61]:
cm = confusion_matrix(y_val, y_pred)
print(cm)
report = classification_report(y_val, y_pred)
print (report) #for each class prints: precision  recall  f1-score   support

[[24957   182]
 [ 1989   386]]
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     25139
           1       0.68      0.16      0.26      2375

   micro avg       0.92      0.92      0.92     27514
   macro avg       0.80      0.58      0.61     27514
weighted avg       0.90      0.92      0.90     27514



In [62]:
probas_ = rfc.fit(X_train, y_train).predict_proba(X_val)
fpr, tpr, thresholds = roc_curve(y_val, probas_[:, 1])
roc_auc = auc(fpr, tpr)
print ("probas_", probas_)
print ("AUC using predict_proba", roc_auc)

probas_ [[0.  1. ]
 [1.  0. ]
 [1.  0. ]
 ...
 [0.9 0.1]
 [1.  0. ]
 [1.  0. ]]
AUC using predict_proba 0.8154648198123694


# Handling missing values : icu_df3 : OneHot Encoding: pd_dummies

In [63]:
icu_df3 = icu_df.copy(deep=True)
icu_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Columns: 186 entries, encounter_id to apache_2_bodysystem
dtypes: float64(170), int64(8), object(8)
memory usage: 130.1+ MB


In [64]:
icu_df3.loc[icu_df3['pre_icu_los_days'] <= 0, 'pre_icu_los_days'] = np.NaN
icu_df3.loc[icu_df3['apache_4a_hospital_death_prob'] == -1, 'apache_4a_hospital_death_prob'] = np.NaN
icu_df3.loc[icu_df3['apache_4a_icu_death_prob'] == -1, 'apache_4a_icu_death_prob'] = np.NaN
# icu_df3.replace('', np.nan, inplace=True)

In [65]:
# OneHotLabel Encoding
icu_df3 = pd.get_dummies(icu_df3, prefix=['ethnicity'], columns=['ethnicity'])
icu_df3 = pd.get_dummies(icu_df3, prefix=['gender'], columns=['gender'])
icu_df3 = pd.get_dummies(icu_df3, prefix=['hospital_admit_source'], columns=['hospital_admit_source'])
icu_df3 = pd.get_dummies(icu_df3, prefix=['icu_admit_source'], columns=['icu_admit_source'])
icu_df3 = pd.get_dummies(icu_df3, prefix=['icu_stay_type'], columns=['icu_stay_type'])
icu_df3 = pd.get_dummies(icu_df3, prefix=['icu_type'], columns=['icu_type'])
# icu_df3 = pd.get_dummies(icu_df3, prefix=['apache_2_diagnosis'], columns=['apache_2_diagnosis'])
# icu_df3 = pd.get_dummies(icu_df3, prefix=['apache_3j_diagnosis'], columns=['apache_3j_diagnosis'])
icu_df3 = pd.get_dummies(icu_df3, prefix=['apache_3j_bodysystem'], columns=['apache_3j_bodysystem'])
icu_df3 = pd.get_dummies(icu_df3, prefix=['apache_2_bodysystem'], columns=['apache_2_bodysystem'])
icu_df3.head(10)

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,...,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M,hospital_admit_source_Acute Care/Floor,hospital_admit_source_Chest Pain Center,hospital_admit_source_Direct Admit,hospital_admit_source_Emergency Department,hospital_admit_source_Floor,hospital_admit_source_ICU,hospital_admit_source_ICU to SDU,hospital_admit_source_Observation,hospital_admit_source_Operating Room,hospital_admit_source_Other,hospital_admit_source_Other Hospital,hospital_admit_source_Other ICU,hospital_admit_source_PACU,hospital_admit_source_Recovery Room,hospital_admit_source_Step-Down Unit (SDU),icu_admit_source_Accident & Emergency,icu_admit_source_Floor,icu_admit_source_Operating Room / Recovery,icu_admit_source_Other Hospital,icu_admit_source_Other ICU,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,apache_3j_bodysystem_Cardiovascular,apache_3j_bodysystem_Gastrointestinal,apache_3j_bodysystem_Genitourinary,apache_3j_bodysystem_Gynecological,apache_3j_bodysystem_Hematological,apache_3j_bodysystem_Metabolic,apache_3j_bodysystem_Musculoskeletal/Skin,apache_3j_bodysystem_Neurological,apache_3j_bodysystem_Respiratory,apache_3j_bodysystem_Sepsis,apache_3j_bodysystem_Trauma,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma

In [66]:
nan_values = icu_df3.isna()
nan_columns = nan_values.any()
columns_with_nan = icu_df3.columns[nan_columns].tolist()
columns_with_nan

['age',
 'bmi',
 'height',
 'pre_icu_los_days',
 'weight',
 'albumin_apache',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'arf_apache',
 'bilirubin_apache',
 'bun_apache',
 'creatinine_apache',
 'fio2_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'intubated_apache',
 'map_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'ventilated_apache',
 'wbc_apache',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_resprate_max',
 'd1_resprate_min',
 'd1_spo2_max',
 'd1_spo2_min',
 'd1_sysbp_invasive_m

In [67]:
icu_df3.dtypes

encounter_id                                 int64
patient_id                                   int64
hospital_id                                  int64
hospital_death                               int64
age                                        float64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary      uint8
apache_2_bodysystem_Respiratory              uint8
apache_2_bodysystem_Trauma                   uint8
apache_2_bodysystem_Undefined Diagnoses      uint8
apache_2_bodysystem_Undefined diagnoses      uint8
Length: 238, dtype: object

In [68]:
obj_cols = icu_df3.select_dtypes(include=['object'])
nan_values = icu_df3.isna()
nan_columns = nan_values.any()
columns_with_nan = icu_df3.columns[nan_columns].tolist()
columns_with_nan

['age',
 'bmi',
 'height',
 'pre_icu_los_days',
 'weight',
 'albumin_apache',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'arf_apache',
 'bilirubin_apache',
 'bun_apache',
 'creatinine_apache',
 'fio2_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'intubated_apache',
 'map_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'ventilated_apache',
 'wbc_apache',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_resprate_max',
 'd1_resprate_min',
 'd1_spo2_max',
 'd1_spo2_min',
 'd1_sysbp_invasive_m

In [69]:
#assoc_matrix = nominal.associations(icu_df3, return_results=True, plot=False,nominal_columns=['hospital_death'])
#print(assoc_matrix.shape)

In [70]:
# fig, ax = plt.subplots(figsize=(200,200))
# sns.heatmap(assoc_matrix, annot=True, ax=ax)

In [71]:
assoc_matrix.loc[:, 'hospital_death'] >= 0.5

NameError: name 'assoc_matrix' is not defined

d1_lactate_max, d1_lactate_min, apache_4a_hospital_death_prob, apache_4a_icu_death_prob have highest association (>0.5) with hospital_death

In [72]:
# type(assoc_matrix)
# assoc_matrix.to_excel("assoc_matrix.xlsx")

In [73]:
X_col_dropped = ['encounter_id','patient_id','hospital_id','icu_id','hospital_death','bmi','readmission_status']
X = icu_df3.drop(labels=X_col_dropped, axis=1)
X = X.select_dtypes(exclude=['object'])
print(X.dtypes)
X_col_names = X.columns.values
X_col_names

age                                        float64
elective_surgery                             int64
height                                     float64
pre_icu_los_days                           float64
weight                                     float64
                                            ...   
apache_2_bodysystem_Renal/Genitourinary      uint8
apache_2_bodysystem_Respiratory              uint8
apache_2_bodysystem_Trauma                   uint8
apache_2_bodysystem_Undefined Diagnoses      uint8
apache_2_bodysystem_Undefined diagnoses      uint8
Length: 231, dtype: object


array(['age', 'elective_surgery', 'height', 'pre_icu_los_days', 'weight',
       'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis',
       'apache_post_operative', 'arf_apache', 'bilirubin_apache',
       'bun_apache', 'creatinine_apache', 'fio2_apache',
       'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache',
       'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache',
       'hematocrit_apache', 'intubated_apache', 'map_apache',
       'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache',
       'resprate_apache', 'sodium_apache', 'temp_apache',
       'urineoutput_apache', 'ventilated_apache', 'wbc_apache',
       'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min',
       'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max',
       'd1_diasbp_noninvasive_min', 'd1_heartrate_max',
       'd1_heartrate_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min',
       'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max',
       'd1_mbp_noninva

In [74]:
y = icu_df3.loc[:,'hospital_death']
print(y.unique())
y

[0 1]


0        0
1        0
2        0
3        0
4        0
        ..
91708    0
91709    0
91710    0
91711    0
91712    0
Name: hospital_death, Length: 91713, dtype: int64

Train & Val split (70:30)

In [75]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))

64199 64199
27514 27514


In [76]:
#rfc = RandomForestClassifier(n_jobs=-1)
#fitted_model = rfc.fit(X_train, y_train)
#y_pred = fitted_model.predict(X_val)
#pTot = accuracy_score(y_true=y_val, y_pred=y_pred)
#pTot

In [77]:
#cm = confusion_matrix(y_val, y_pred)
#print(cm)
#report = classification_report(y_val, y_pred)
#print (report) #for each class prints: precision  recall  f1-score   support

# importance plot with all variables

In [78]:
icu_df4 = icu_df.copy(deep=True)

In [79]:
le = LabelEncoder()
icu_df4['ethnicity'] = le.fit_transform(icu_df4['ethnicity'].astype(str))
icu_df4['gender'] = le.fit_transform(icu_df4['gender'].astype(str))
icu_df4['hospital_admit_source'] = le.fit_transform(icu_df4['hospital_admit_source'].astype(str))
icu_df4['icu_admit_source'] = le.fit_transform(icu_df4['icu_admit_source'].astype(str))
icu_df4['icu_stay_type'] = le.fit_transform(icu_df4['icu_stay_type'].astype(str))
icu_df4['icu_type'] = le.fit_transform(icu_df4['icu_type'].astype(str))
icu_df4['apache_3j_bodysystem'] = le.fit_transform(icu_df4['apache_3j_bodysystem'].astype(str))
icu_df4['apache_2_bodysystem'] = le.fit_transform(icu_df4['apache_2_bodysystem'].astype(str))
icu_df4['apache_4a_icu_death_prob'] = le.fit_transform(icu_df4['apache_4a_icu_death_prob'].astype(str))

icu_df4.head(5)

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,2,1,180.3,4,1,92,0,2,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9,0
1,114252,59342,81,0,77.0,27.42,0,2,0,160.0,4,1,90,0,5,0.927778,0,70.2,NaN,108.0,203.01,0,0.0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0.0,46.0

In [80]:
icu_df4.loc[icu_df4['pre_icu_los_days'] <= 0, 'pre_icu_los_days'] = np.NaN
icu_df4.loc[icu_df4['apache_4a_hospital_death_prob'] == -1, 'apache_4a_hospital_death_prob'] = np.NaN

In [81]:
X_col_dropped = ['encounter_id', 'patient_id', 'hospital_id','hospital_death', 'icu_id', 'readmission_status']
X_train_val = icu_df4.drop(labels=X_col_dropped, axis=1)
#print(X.dtypes)
X_col_names = X_train_val.columns.values
X_col_names

array(['age', 'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height',
       'hospital_admit_source', 'icu_admit_source', 'icu_stay_type',
       'icu_type', 'pre_icu_los_days', 'weight', 'albumin_apache',
       'apache_2_diagnosis', 'apache_3j_diagnosis',
       'apache_post_operative', 'arf_apache', 'bilirubin_apache',
       'bun_apache', 'creatinine_apache', 'fio2_apache',
       'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache',
       'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache',
       'hematocrit_apache', 'intubated_apache', 'map_apache',
       'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache',
       'resprate_apache', 'sodium_apache', 'temp_apache',
       'urineoutput_apache', 'ventilated_apache', 'wbc_apache',
       'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min',
       'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max',
       'd1_diasbp_noninvasive_min', 'd1_heartrate_max',
       'd1_heartrate_min', 'd1_mbp_inv

In [82]:
y_train_val = icu_df4.loc[:,'hospital_death']
print(y_train_val.unique())
y_train_val

[0 1]


0        0
1        0
2        0
3        0
4        0
        ..
91708    0
91709    0
91710    0
91711    0
91712    0
Name: hospital_death, Length: 91713, dtype: int64

In [83]:
# imputing Nan's with mode in X_train_val and X_tests
imp_x = SimpleImputer(missing_values=np.nan, strategy='median') 
X_train_val_imp = imp_x.fit_transform(X_train_val)
# X_test = imp_x.transform(X_test)

In [84]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val_imp, y_train_val, test_size=0.3, random_state=1, stratify=y_train_val)
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))

64199 64199
27514 27514


In [85]:
rfc = RandomForestClassifier(n_jobs=-1)
fitted_model = rfc.fit(X_train, y_train)
y_pred = fitted_model.predict(X_val)
pTot = accuracy_score(y_true=y_val, y_pred=y_pred)
pTot

/Users/priyankasharma/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



0.9272370429599477

In [97]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_train_val.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(70)

,importance
apache_4a_hospital_death_prob,0.101435
d1_lactate_min,0.033879
d1_sysbp_min,0.029691
d1_lactate_max,0.027069
d1_temp_max,0.021549
d1_spo2_min,0.020651
d1_mbp_min,0.017701
temp_apache,0.016226
bmi,0.016013
heart_rate_apache,0.016002


# importance plot with xgboost_2 file variables

In [87]:
icu_df5 = icu_df.copy(deep=True)

In [88]:
icu_df5 = icu_df5.drop(labels=['encounter_id', 'hospital_id', 'patient_id', 'elective_surgery', 'height', 'icu_id', 'weight', 'gcs_eyes_apache', 'gcs_verbal_apache', 'glucose_apache', 
'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_heartrate_max', 'd1_heartrate_min', 
'd1_mbp_invasive_max', 'd1_mbp_invasive_min', 'd1_mbp_noninvasive_max','d1_mbp_noninvasive_min', 'd1_sysbp_invasive_max', 'd1_sysbp_invasive_min', 
'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'h1_diasbp_invasive_max', 'h1_diasbp_invasive_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_diasbp_noninvasive_max',
 'h1_diasbp_noninvasive_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_invasive_max', 'h1_mbp_invasive_min', 'h1_mbp_max', 'h1_mbp_min', 'h1_mbp_noninvasive_max', 
 'h1_mbp_noninvasive_min', 'h1_spo2_max', 'h1_spo2_min', 'h1_sysbp_invasive_max', 'h1_sysbp_invasive_min', 'h1_sysbp_max', 'h1_sysbp_min', 'h1_sysbp_noninvasive_max', 
 'h1_sysbp_noninvasive_min', 'h1_temp_max', 'h1_temp_min', 'd1_albumin_max', 'd1_albumin_min', 'd1_bilirubin_max', 'd1_bilirubin_min', 'd1_hematocrit_max', 
 'd1_hematocrit_min', 'd1_inr_min', 'h1_albumin_max', 'h1_albumin_min', 'h1_bilirubin_max', 'h1_bilirubin_min', 'h1_bun_max', 'h1_bun_min', 'h1_calcium_max', 
 'h1_calcium_min', 'h1_creatinine_max', 'h1_creatinine_min', 'h1_glucose_max', 'h1_glucose_min', 'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max', 'h1_hemaglobin_min', 
 'h1_hematocrit_max', 'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min', 'h1_lactate_max', 'h1_lactate_min', 'h1_platelets_max', 'h1_platelets_min', 'h1_potassium_max', 
 'h1_potassium_min', 'h1_sodium_max', 'h1_sodium_min', 'h1_wbc_max', 'h1_wbc_min', 'd1_arterial_ph_min', 'd1_pao2fio2ratio_max', 'd1_pao2fio2ratio_min', 'h1_arterial_pco2_max',
'h1_arterial_pco2_min', 'h1_arterial_ph_max', 'h1_arterial_ph_min', 'h1_arterial_po2_max', 'h1_arterial_po2_min', 'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 
'apache_4a_icu_death_prob', 'apache_3j_diagnosis'], axis=1)

In [89]:
# label encoding
le = LabelEncoder()
icu_df5['ethnicity'] = le.fit_transform(icu_df5['ethnicity'].astype(str))
icu_df5['gender'] = le.fit_transform(icu_df5['gender'].astype(str))
icu_df5['hospital_admit_source'] = le.fit_transform(icu_df5['hospital_admit_source'].astype(str))
icu_df5['icu_admit_source'] = le.fit_transform(icu_df5['icu_admit_source'].astype(str))
icu_df5['icu_stay_type'] = le.fit_transform(icu_df5['icu_stay_type'].astype(str))
icu_df5['icu_type'] = le.fit_transform(icu_df5['icu_type'].astype(str))
icu_df5['apache_3j_bodysystem'] = le.fit_transform(icu_df5['apache_3j_bodysystem'].astype(str))
icu_df5['apache_2_bodysystem'] = le.fit_transform(icu_df5['apache_2_bodysystem'].astype(str))
icu_df5.head(5)

,hospital_death,age,bmi,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,albumin_apache,apache_2_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_motor_apache,gcs_unable_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_mbp_max,d1_mbp_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,h1_resprate_max,h1_resprate_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_inr_max,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_po2_max,d1_arterial_po2_min,apache_4a_hospital_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,0,68.0,22.73,2,1,4,1,0,2,0.541667,0,2.3,113.0,0,0.0,0.4,31.0,2.51,NaN,6.0,0.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,68.0,37.0,89.0,46.0,34.0,10.0,100.0,74.0,131.0,73.0,39.9,37.2,26.0,18.0,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,0.10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9,0
1,0,77.0,27.42,2,0,4,1,0,5,0.927778,0,NaN,108.0,0,0.0,NaN,9.0,0.56,1.0,3.0,0.0,120.0,36.9,0.0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1.0,12.7,95.0,31.0,120.0,38.0,32.0,12.0,100.0,70.0,159.0,67.0,36.3,35.1,31.0,28.0,11.0,9.0,8.6,8.0,0.71,0.56,145.0,128.0,27.0,26.0,11.3,11.1,1.3,3.5,3.5,557.0,487.0,4.2,3.8,145.0,145.0,23.3,12.7,37.0,37.0,7.45,51.0,51.0,0.47,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8,6
2,0,25.0,31.95,2,0,3,0,0,5,0.000694,0,NaN,122.0,0,0.0,NaN,NaN,NaN,NaN,6.0,0.0,102.0,NaN,0.0,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,NaN,0.0,NaN,88.0,48.0,102.0,68.0,21.0,8.0,98.0,91.0,148.0,105.0,37.0,36.7,20.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,3
3,0,81.0,22.64,2,0,8,2,0,2,0.000694,0,NaN,203.0,1,0.0,NaN,NaN,NaN,0.6,6.0,0.0,114.0,25.9,1.0,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,NaN,1.0,8.0,48.0,42.0,84.0,84.0,23.0,7.0,100.0,95.0,158.0,84.0,38.0,34.8,12.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,185.0,88.0,NaN,NaN,11.6,8.9,1.6,NaN,NaN,198.0,43.0,5.0,3.5,NaN,NaN,9.0,8.0,37.0,27.0,7.44,337.0,102.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,0,19.0,NaN,2,1,15,0,0,5,0.073611,0,NaN,119.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0.0,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,NaN,0.0,NaN,99.0,57.0,104.0,90.0,18.0,16.0,100.0,96.0,147.0,120.0,37.2,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,7


In [90]:
# replace impossible values
icu_df5.loc[icu_df5['pre_icu_los_days'] <= 0, 'pre_icu_los_days'] = np.NaN
icu_df5.loc[icu_df5['apache_4a_hospital_death_prob'] == -1, 'apache_4a_hospital_death_prob'] = np.NaN

In [91]:
X_col_dropped = ['hospital_death']
X_train_val = icu_df5.drop(labels=X_col_dropped, axis=1)
#print(X.dtypes)
X_col_names = X_train_val.columns.values
X_col_names

array(['age', 'bmi', 'ethnicity', 'gender', 'hospital_admit_source',
       'icu_admit_source', 'icu_stay_type', 'icu_type',
       'pre_icu_los_days', 'readmission_status', 'albumin_apache',
       'apache_2_diagnosis', 'apache_post_operative', 'arf_apache',
       'bilirubin_apache', 'bun_apache', 'creatinine_apache',
       'fio2_apache', 'gcs_motor_apache', 'gcs_unable_apache',
       'heart_rate_apache', 'hematocrit_apache', 'intubated_apache',
       'map_apache', 'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache',
       'ph_apache', 'resprate_apache', 'sodium_apache', 'temp_apache',
       'urineoutput_apache', 'ventilated_apache', 'wbc_apache',
       'd1_diasbp_max', 'd1_diasbp_min', 'd1_mbp_max', 'd1_mbp_min',
       'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min',
       'd1_sysbp_max', 'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min',
       'h1_resprate_max', 'h1_resprate_min', 'd1_bun_max', 'd1_bun_min',
       'd1_calcium_max', 'd1_calcium_min', 'd1_cre

In [92]:
y_train_val = icu_df5.loc[:,'hospital_death']
print(y_train_val.unique())
y_train_val

[0 1]


0        0
1        0
2        0
3        0
4        0
        ..
91708    0
91709    0
91710    0
91711    0
91712    0
Name: hospital_death, Length: 91713, dtype: int64

In [93]:
# imputing Nan's with mode in X_train_val and X_tests
imp_x = SimpleImputer(missing_values=np.nan, strategy='median') 
X_train_val_imp = imp_x.fit_transform(X_train_val)
# X_test = imp_x.transform(X_test)

In [94]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val_imp, y_train_val, test_size=0.3, random_state=1, stratify=y_train_val)
print(len(X_train),len(y_train))
print(len(X_val),len(y_val))

64199 64199
27514 27514


In [100]:
# rf
#rfc = RandomForestClassifier(n_jobs=-1)
from xgboost import XGBClassifier
model = XGBClassifier(scale_pos_weight = 10.6)
fitted_model = model.fit(X_train, y_train)
y_pred = fitted_model.predict(X_val)
pTot = accuracy_score(y_true=y_val, y_pred=y_pred)
pTot

0.816493421530857

In [101]:
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = X_train_val.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(70)

,importance
apache_4a_hospital_death_prob,0.174004
ventilated_apache,0.056189
d1_sysbp_min,0.040997
d1_resprate_max,0.033105
d1_bun_max,0.030513
d1_bun_min,0.030494
d1_spo2_min,0.028336
d1_arterial_po2_min,0.027131
d1_lactate_min,0.025886
h1_resprate_min,0.024257
